<a href="https://colab.research.google.com/github/ximo2001/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2)_Ximo_Sanz_Tornero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Ximo Sanz Tornero  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1Z2I54KbRyM-TG635pwkFTTxF20VwyZvF <br>
Problema: <br>
Organizar los horarios de partidos de La Liga <br>
Descripción del problema: Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la
audiencia.









                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?


1.   El espacio de soluciones está formado por todos los vectores en los que, para cada partido, se asigna una franja horaria específica. Por ejemplo, si hay n partidos y m franjas horarias, una solución se puede representar como un vector de longitud n, donde cada posición indica la franja horaria (índice entre 0 y m−1) elegida para ese partido. En este caso el espacio de soluciones será el conjunto de todos los vectores de la forma [s1, s2, ..., s10], en el que cada componente es un entero entre 0 y 9 que corresponde a la franja horaria.  
2.   La función objetivo se calcula sumando, para cada partido, la audiencia efectiva que depende de:  
• La audiencia base de los dos equipos enfrentados.  
• Un coeficiente multiplicador según el día y la hora de la franja.  
• Una penalización por coincidencias si en esa franja horaria hay varios partidos.  
Así, para cada partido i asignado a la franja j, se determina la audiencia base (según la categoría de los equipos), se multiplica por el coeficiente del día/hora y luego se aplica (1 − penalización) si en esa franja hay partidos coincidentes. La suma de todas las audiencias efectivas da la función objetivo que se desea maximizar.  
3. En cuanto a las restricciones del problema se exige que al menos un partido ocurra el viernes a las 20 y al menos uno en lunes a las 8. Si la solución proporcionada no cumple con estas restricciones la función obtetivo devuelve menos infinito, descartando esa opción.



#Análisis

- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

El tamaño del espacio de soluciones es exponencial porque cada partido puede asignarse a cualquiera de las m franjas horarias, generando en total m^n combinaciones.  Al aumentar un partido adicional, se multiplica automáticamente por m el total de posibles asignaciones, lo cual hace crecer el número de soluciones de manera explosiva.  En este caso con 10 partidos y 10 franjas horarias, se tienen 10^10 = 10 mil millones de soluciones distintas.  


  #Diseño
  - ¿Que técnica utilizo? ¿Por qué?

Se optó por utilizar **Recocido Simulado (Simulated Annealing)** tras evaluar varias alternativas, como la Búsqueda Tabú, algoritmos Genéticos y combinaciones de algoritmos Genéticos con enfriamiento. En múltiples pruebas, el Recocido Simulado no solo produjo soluciones de mayor calidad, sino que además demostró ser muy eficiente en cuanto a tiempo de cómputo para este problema en particular.

En esta técnica, se imita el proceso de enfriamiento de un material: al inicio se permite una exploración amplia del espacio de soluciones, aceptando ocasionalmente soluciones peores para evitar quedarse atrapado en óptimos locales. Conforme la "temperatura" disminuye, la búsqueda se concentra en refinar las mejores soluciones encontradas, lo que resulta en una optimización gradual y controlada.

En comparación, la Búsqueda Tabú, aunque efectiva para evitar ciclos y explorar extensos vecindarios, requiere una gestión compleja de movimientos prohibidos y puede demandar más tiempo de cómputo. Los algoritmos Genéticos, por su parte, al trabajar con poblaciones de soluciones y aplicar operadores de cruce y mutación, enfrentan mayores dificultades para manejar restricciones específicas del problema (como el máximo de partidos por franja o franjas obligatorias), lo que en ocasiones conlleva a soluciones de menor calidad o a tiempos de procesamiento más largos.

La eficiencia del Recocido Simulado en este caso se debe a su capacidad para equilibrar la exploración y explotación del espacio de soluciones de manera natural. Al penalizar de forma inmediata las soluciones que violan las restricciones, la técnica se concentra en las regiones factibles del problema sin la necesidad de complicados mecanismos de reparación. Este enfoque, junto con su sencilla implementación y ajuste de parámetros, permitió obtener resultados superiores en menor tiempo.


In [ ]:
#Respuesta
import random
import math

# ---------------------------
# Parámetros globales del problema
# ---------------------------

# Fijamos la semilla para reproducibilidad
random.seed(42)

# Franjas horarias disponibles (Día, Hora)
time_slots = [
    ("Viernes", 20),
    ("Sábado", 12),
    ("Sábado", 16),
    ("Sábado", 18),
    ("Sábado", 20),
    ("Domingo", 12),
    ("Domingo", 16),
    ("Domingo", 18),
    ("Domingo", 20),
    ("Lunes", 20)
]

# Coeficientes según el día y la hora
time_coeff = {
    ("Viernes", 20): 0.4,
    ("Sábado", 12): 0.55,
    ("Sábado", 16): 0.7,
    ("Sábado", 18): 0.8,
    ("Sábado", 20): 1.0,
    ("Domingo", 12): 0.45,
    ("Domingo", 16): 0.75,
    ("Domingo", 18): 0.85,
    ("Domingo", 20): 1.0,
    ("Lunes", 20): 0.4
}

# Penalizaciones por coincidencia:
# "coincidencias" = (cantidad de partidos en la franja) - 1
coincidence_penalty = {
    0: 0.0,
    1: 0.25,
    2: 0.45,
    3: 0.60,
    4: 0.70,
    5: 0.75,
    6: 0.78,
    7: 0.80,
    8: 0.80
}

# Máximo de partidos permitidos en una misma franja, esta restricción la implemento puesto que la cantidad de audiencia perdida hace que soluciones como esta sean malas
max_partidos_por_franja = 5

# Tabla de audiencia base (en millones) para partidos en el mejor horario (Sábado 20)
# Se asume simetría: (A, B) = (B, A)
base_audience = {
    ('A', 'A'): 2.0,
    ('A', 'B'): 1.3,
    ('A', 'C'): 1.0,
    ('B', 'B'): 0.9,
    ('B', 'C'): 0.75,
    ('C', 'C'): 0.47
}

# Índices de franjas obligatorias (al menos un partido en cada una)
index_viernes = time_slots.index(("Viernes", 20))
index_lunes   = time_slots.index(("Lunes", 20))

def get_base_audience(cat1, cat2):
    """
    Retorna la audiencia base de un partido entre cat1 y cat2,
    asumiendo simetría en la tabla.
    """
    key = tuple(sorted((cat1, cat2)))
    return base_audience.get(key, 0)

# ---------------------------
# Generación de equipos y partidos
# ---------------------------
def generate_teams():
    """
    Genera la lista de 20 equipos con sus categorías:
      3 equipos de tipo A
      11 equipos de tipo B
      6 equipos de tipo C
    """
    teams = (['A'] * 3) + (['B'] * 11) + (['C'] * 6)
    return teams

def generate_matches():
    """
    Genera aleatoriamente 10 partidos (pares) a partir de 20 equipos.
    Cada partido se representa como (cat_equipo1, cat_equipo2).
    """
    teams = generate_teams()
    random.shuffle(teams)  # Desordena la lista de equipos con la semilla fijada
    matches = []
    # Agrupamos de 2 en 2 para formar 10 partidos
    for i in range(0, len(teams), 2):
        cat1 = teams[i]
        cat2 = teams[i+1]
        matches.append((cat1, cat2))
    return matches

# ---------------------------
# Función de evaluación
# ---------------------------
def evaluate_solution(solution, matches):
    """
    Calcula la audiencia total de la asignación de partidos a franjas.
    solution: lista con un índice de franja por cada partido.
    matches: lista de partidos, cada uno (cat_equipo1, cat_equipo2).
    Retorna -inf si se incumplen las restricciones.
    """
    from math import inf

    # Conteo de partidos en cada franja
    counts = {i: 0 for i in range(len(time_slots))}
    for slot_index in solution:
        counts[slot_index] += 1

    # Verifica restricción de máximo partidos por franja
    for count in counts.values():
        if count > max_partidos_por_franja:
            return -inf

    # Verifica que haya al menos un partido en viernes y en lunes
    if counts[index_viernes] < 1 or counts[index_lunes] < 1:
        return -inf

    # Calcula penalizaciones por coincidencia
    franja_penalty = {}
    for idx, cnt in counts.items():
        coincidencias = cnt - 1 if cnt > 0 else 0
        pen = coincidence_penalty.get(coincidencias, max(coincidence_penalty.values()))
        franja_penalty[idx] = pen

    # Suma de audiencias
    total_audience = 0.0
    for i, slot_index in enumerate(solution):
        dia_hora = time_slots[slot_index]
        coeff = time_coeff[dia_hora]
        base_val = get_base_audience(matches[i][0], matches[i][1])
        pen = franja_penalty[slot_index]
        effective = base_val * coeff * (1 - pen)
        total_audience += effective

    return total_audience

# ---------------------------
# Metaheurística: Simulated Annealing
# ---------------------------
def solution_search(matches, max_iter=10000, initial_temp=2.0, cooling_rate=0.999):
    """
    Realiza la búsqueda de la mejor asignación de partidos (Simulated Annealing).
    matches: lista de partidos.
    max_iter: número máximo de iteraciones.
    initial_temp: temperatura inicial.
    cooling_rate: factor de enfriamiento.

    Retorna (best_solution, best_cost).
    """
    from math import exp

    n_matches = len(matches)
    n_slots = len(time_slots)

    # Solución inicial aleatoria
    current_solution = [random.randint(0, n_slots - 1) for _ in range(n_matches)]
    current_cost = evaluate_solution(current_solution, matches)

    best_solution = current_solution[:]
    best_cost = current_cost

    temperature = initial_temp

    for _ in range(max_iter):
        # Generamos una solución vecina: cambiar el slot de un partido al azar
        neighbor = current_solution[:]
        match_to_move = random.randint(0, n_matches - 1)
        new_slot = random.randint(0, n_slots - 1)
        neighbor[match_to_move] = new_slot

        neighbor_cost = evaluate_solution(neighbor, matches)

        # Criterio de aceptación
        cost_diff = neighbor_cost - current_cost
        if cost_diff > 0:
            current_solution = neighbor
            current_cost = neighbor_cost
        else:
            if random.random() < math.exp(cost_diff / temperature):
                current_solution = neighbor
                current_cost = neighbor_cost

        # Actualización de la mejor solución global
        if current_cost > best_cost:
            best_solution = current_solution[:]
            best_cost = current_cost

        # Enfriamiento
        temperature *= cooling_rate
        if temperature < 1e-6:
            temperature = 1e-6

    if best_cost == float('-inf'):
        return None, best_cost
    else:
        return best_solution, best_cost

# ---------------------------
# Función para probar múltiples parámetros
# ---------------------------
def try_different_parameters(matches, param_list):
    """
    Ejecuta solution_search con distintas configuraciones y retorna
    la mejor solución global entre todas.
    param_list: lista de tuplas (max_iter, initial_temp, cooling_rate).

    Retorna (best_solution, best_cost, best_params), donde best_params
    es la tupla de parámetros que produjo el mejor resultado.
    """
    best_global_solution = None
    best_global_cost = float('-inf')
    best_params = None

    for params in param_list:
        max_iter, init_temp, cool_rate = params
        # Llamamos a la metaheurística con estos parámetros
        sol, cost = solution_search(matches,
                                    max_iter=max_iter,
                                    initial_temp=init_temp,
                                    cooling_rate=cool_rate)
        # Si es la mejor solución hasta ahora, la guardamos
        if cost > best_global_cost:
            best_global_cost = cost
            best_global_solution = sol
            best_params = params

    return best_global_solution, best_global_cost, best_params

# ---------------------------
# Programa principal
# ---------------------------
if __name__ == '__main__':
    # Generamos los partidos a partir de 20 equipos
    matches = generate_matches()

    print("Buscando la mejor asignación de partidos a franjas...")

    # Definimos un conjunto de parámetros a explorar
    # Puedes añadir más combinaciones según se requiera
    parameter_configurations = [
        (5000, 2.0, 0.999),
        (10000, 2.0, 0.999),
        (5000, 3.0, 0.999),
        (5000, 2.0, 0.9),
        (10000, 2.0, 0.9),
        (5000, 3.0, 0.9),
        (5000, 2.0, 0.998)
    ]

    # Probamos distintas configuraciones y obtenemos la mejor
    best_sol, best_aud, best_par = try_different_parameters(matches, parameter_configurations)

    if best_sol is None or best_aud == float('-inf'):
        print("No se encontró ninguna solución válida.")
    else:
        print(f"\nMejor solución encontrada con audiencia total: {best_aud:.3f} millones")
        print(f"Parámetros usados: max_iter={best_par[0]}, initial_temp={best_par[1]}, cooling_rate={best_par[2]}")

        # Recontamos cuántos partidos hay en cada franja
        counts = {i: 0 for i in range(len(time_slots))}
        for slot_index in best_sol:
            counts[slot_index] += 1

        print("\nDetalle de asignaciones (por partido):")
        for i, slot_index in enumerate(best_sol):
            dia, hora = time_slots[slot_index]
            coeff = time_coeff[(dia, hora)]
            base_val = get_base_audience(matches[i][0], matches[i][1])
            pen = coincidence_penalty.get(counts[slot_index] - 1, max(coincidence_penalty.values()))
            effective = base_val * coeff * (1 - pen)
            print(f"  Partido {i+1} ({matches[i][0]} vs {matches[i][1]}): "
                  f"{dia} {hora}h, base={base_val:.2f}, coeff={coeff:.2f}, "
                  f"coincidencias={counts[slot_index]-1}, penalización={pen*100:.0f}%, "
                  f"audiencia={effective:.2f}M")

        print("\nResumen de franjas horarias:")
        for idx, (dia, hora) in enumerate(time_slots):
            print(f"  {dia} {hora}h: {counts[idx]} partido(s)")


Buscando la mejor asignación de partidos a franjas...

Mejor solución encontrada con audiencia total: 6.666 millones
Parámetros usados: max_iter=10000, initial_temp=2.0, cooling_rate=0.999

Detalle de asignaciones (por partido):
  Partido 1 (C vs B): Viernes 20h, base=0.75, coeff=0.40, coincidencias=0, penalización=0%, audiencia=0.30M
  Partido 2 (C vs B): Lunes 20h, base=0.75, coeff=0.40, coincidencias=0, penalización=0%, audiencia=0.30M
  Partido 3 (B vs B): Domingo 18h, base=0.90, coeff=0.85, coincidencias=0, penalización=0%, audiencia=0.77M
  Partido 4 (C vs C): Sábado 12h, base=0.47, coeff=0.55, coincidencias=0, penalización=0%, audiencia=0.26M
  Partido 5 (B vs B): Sábado 16h, base=0.90, coeff=0.70, coincidencias=0, penalización=0%, audiencia=0.63M
  Partido 6 (C vs B): Domingo 12h, base=0.75, coeff=0.45, coincidencias=0, penalización=0%, audiencia=0.34M
  Partido 7 (A vs B): Domingo 20h, base=1.30, coeff=1.00, coincidencias=0, penalización=0%, audiencia=1.30M
  Partido 8 (A vs C